In [ ]:
from data import data
import pandas as pd
import numpy as np
from analyze import analyze
from plots import scatter, trend, lmh, trend_scatter
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt
from Freedom.freedom import freedom

In [ ]:
analyze(data, 2017)

In [ ]:
scatter(data, 2017, 'GDP per capita (current US$)',  'Population, total')

In [ ]:
trend(data, ["World", "Brazil", "Mexico"], 'GDP per capita (current US$)', 1963, 2017)

In [ ]:
lmh(data, 2017, 'GDP per capita (current US$)', 2)

In [ ]:
trend_scatter(data, "Brazil", 'Population, total', 2000, 2005, True)

In [ ]:
gdp = trend_scatter(data, "World", 'GDP per capita (current US$)', 2008, 2014, False)
for i in analyze(data, 2014).columns[1:].drop(["Net barter terms of trade index (2000 = 100)", "New businesses registered (number)","Official exchange rate (LCU per US$, period average)","Current account balance (BoP, current US$)", "Real interest rate (%)","Lending interest rate (%)","Total reserves (includes gold, current US$)","IBRD loans and IDA credits (DOD, current US$)","External debt stocks, total (DOD, current US$)","Use of IMF credit (DOD, current US$)"]):
    isc = scatter(data, 2014, 'GDP per capita (current US$)',  i, False)
    itr = trend_scatter(data, "World", i, 2008, 2014, False)
    if isc[0] > 0.4 and abs(itr[1]-gdp[1]) < 0.1 and isc[2] > 9:
        scatter(data, 2014, 'GDP per capita (current US$)', i)
        trend_scatter(data, "World", i, 2008, 2014)